# Clasificando flores

En nuestra segunda practica abordaremos el problema de la clasificación multiclase. Para ello usaremos otra de las bases de datos clásicas, la *iris dataset* que contiene características de petalos de tres clases distintas.

La clasificación multiclase es un problema interesante puesto que cada clase necesita un tratamiento especializado.

En este caso, los mejores resultados (y los que esperamos obtener) es una exactitud cercana al 95%.

No se hable más y manos a la obra.

Igual que siempre, cargamos las liberías necesarias.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn import datasets
from google.colab import files

In [ ]:
plt.rcParams['figure.figsize'] = (12,12)

## Cargamos la base de datos

La base de datos Iris consta de valores numéricos como datos de entrada, pero, la variable de salida se indica con texto (esto lo veremos más adelante). Esta base ha sido estudiada bastante y es un buen ejercicio para prácticar las NN's por su relativa sencillez en cuanto al manejo de datos. Como mencionamos antes, la base se compone de cuatro instancias de mediciones de flores y una instancia final de la especie a la que pertenece dicha flor.

In [ ]:
# Cargando la base
uploads = files.upload()

Saving iris_dataset.csv to iris_dataset (1).csv


In [ ]:
upload = pd.read_csv('iris_dataset.csv')

In [ ]:
data = pd.DataFrame(upload)
data.head(-1)

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa
...,...,...,...,...,...
144,6.7,3.3,5.7,2.5,Virginica
145,6.7,3.0,5.2,2.3,Virginica
146,6.3,2.5,5.0,1.9,Virginica
147,6.5,3.0,5.2,2.0,Virginica


In [ ]:
data.describe()

,sepal.length,sepal.width,petal.length,petal.width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333
std,0.828066,0.435866,1.765298,0.762238
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [ ]:
database = data.values # Asignamos a la variable database todos los valores de la base de datos
X = database[:,0:4].astype(float) # X serán los valores de nuestras primeras cuatro columnas
Y = database[:,4] # Y serán los valores de la última columna

In [ ]:
np.shape(Y)

(150,)

Como lo mencionamos, la variable de salida es de tipo *string*, por lo que se debe convertir a numérica.

Cuando nos enfrentamos a un problema multiclase, una buena practica es cambiar la forma de los datos de salida. Una solución rápida es crear un vector que contenga valores para cada clase. Esta técnica se llama **one-hot encoding**.

En este caso nuestro vector de salida quedaría de la siguente forma:
* Setosa     --> [1,0,0]
* Versicolor --> [0,1,0]
* Virginica  --> [0,0,1]


In [ ]:
# Encodear los valores de clase a numéricos
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# Convertir los enteros en variables de prueba
dummy_y = np_utils.to_categorical(encoded_Y)

In [ ]:
# Desplegamos los valores después del one-hot encoding
print(dummy_y[0:10])

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


Como pueden observar, ya tenemos solucionado (de forma rápida y elegante) el problema de las variables categóricas.

Ahora vamos a modelar nuestra red.

Ya que hicimos la conversión a valores numéricos es tiempo de generar nuestros conjuntos de entrenamiento y prueba.

Para entrenar la red usaremos el 70% de los datos, para probar su funcionamiento el 30% restante.

In [ ]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(X, dummy_y,
                                                test_size=0.3,random_state=0)

In [ ]:
print(np.shape(Xtrain))
print(np.shape(Xtest))
print(np.shape(ytrain))
print(np.shape(ytest))

(105, 4)
(45, 4)
(105, 3)
(45, 3)


## Definiendo la red neuronal

In [ ]:
model = Sequential()
model.add(Dense(8, input_dim=4, activation='relu')) # capa de entrada
model.add(Dense(16,activation='relu')) # capa oculta
model.add(Dense(3, activation='softmax')) # capa de salida

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 40        
_________________________________________________________________
dense_1 (Dense)              (None, 16)                144       
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 51        
Total params: 235
Trainable params: 235
Non-trainable params: 0
_________________________________________________________________


La arquitectura de la red será algo sencillo.
* Una capa de entrada donde estarán nuestras inputs, esta capa tendrá 8 perceptrones y se activará con la función ReLU
* Una capa de salida, con tres posibles resultados y que se active a través de la función Softmax.
* Como método de evaluación usaremos a la *accuracy*.

In [ ]:
# Viendo de modo gráfico a la red
import tensorflow as tf

tf.keras.utils.plot_model(model, show_shapes=True)

AssertionError: ignored

In [ ]:
# Iniciamos el entrenamiento con el maestro Roshi
model.fit(Xtrain,ytrain, epochs=250,batch_size=32,verbose=1)

Epoch 1/250
4/4 [==============================] - 13s 3ms/step - loss: 1.3348 - accuracy: 0.3736
Epoch 2/250
4/4 [==============================] - 0s 2ms/step - loss: 1.2686 - accuracy: 0.3632
Epoch 3/250
4/4 [==============================] - 0s 2ms/step - loss: 1.1872 - accuracy: 0.3861
Epoch 4/250
4/4 [==============================] - 0s 2ms/step - loss: 1.1938 - accuracy: 0.3777
Epoch 5/250
4/4 [==============================] - 0s 3ms/step - loss: 1.1132 - accuracy: 0.4017
Epoch 6/250
4/4 [==============================] - 0s 3ms/step - loss: 1.1506 - accuracy: 0.3819
Epoch 7/250
4/4 [==============================] - 0s 3ms/step - loss: 1.1573 - accuracy: 0.3413
Epoch 8/250
4/4 [==============================] - 0s 4ms/step - loss: 1.0989 - accuracy: 0.3621
Epoch 9/250
4/4 [==============================] - 0s 3ms/step - loss: 1.1260 - accuracy: 0.3382
Epoch 10/250
4/4 [==============================] - 0s 3ms/step - loss: 1.0297 - accuracy: 0.3975
Epoch 11/250
4/4 [==========

## Evaluando el desempeño de la red

Ya que terminó el entrenamiento debemos evaluar la red.
Para este caso usaremos las métricas de exactitud (accuracy) y el valor de pérdida (loss). Estas métricas son de las más usadas para evaluar una red, además de que las podemos obtener usando la función *model.evaluate()*.

In [ ]:
# loss, accuracy = model.evaluate(Xtest,ytest,steps=3)
loss, accuracy = model.evaluate(Xtest,ytest)
print("Loss: %f" % (loss))
print("Accuracy: %f" % (accuracy))

2/2 [==============================] - 0s 7ms/step - loss: 0.1193 - accuracy: 0.9778
Loss: 0.119314
Accuracy: 0.977778


Con la topología de nuestra red obtenemos un exactitud del 95% en la última época de entrenamiento, también conseguimos una exactitud del 93% en el conjunto de prueba. Nuestro valor de pérdida también es muy bajo. Con estos datos podemos decir que el entrenamiento fue existoso y que nuestra red es muy buena.

### Matriz de confusión para una clasificación multiclase

Si en una clasificación binaria hacer la matriz de confusión es complicado, esperen a ver una matriz multiclase...

Para este caso confiaremos ciegamente en el resultado del valor de pérdida y de exactitud.

## Usando la red para nuevas flores

Vimos que las métricas son muy buenas, pero esto sólo funciona con datos que la red conoce. ¿Qué pasaría si introducimos datos desconocidos?¿Será capaz la red de asignar correctamente una especie? Con la función *model.predict()* podemos hacer esta acción.

In [ ]:
# Definimos nuevas características de una flor
# new_flower = np.array([7.9,3.8,6.4,2.0])
new_flower = np.array([5.1,3.5,1.4,0.2])
prediction = np.around(model.predict(np.expand_dims(new_flower, axis=0))).astype(np.int)[0]
print(prediction)
pred = np.array_str(prediction)
print(pred)
name_dict = {'[1 0 0]':'Setosa','[0 1 0]':'Versicolor','[0 0 1]':'Virginica'}

print("Esta flor debería ser %s" % name_dict[pred])

[1 0 0]
[1 0 0]
Esta flor debería ser Setosa


In [ ]:
name_dict[pred]

'Setosa'

In [ ]:
new_flower = np.array([9.1,5.0,2.6,1.9])
prediction = np.around(model.predict(np.expand_dims(new_flower, axis=0))).astype(np.int)[0]
pred = np.array_str(prediction)
name_dict = {'[1 0 0]':'Setosa','[0 1 0]':'Versicolor','[0 0 1]':'Virginica'}

print("Esta flor debería ser %s" % name_dict[pred])

Esta flor debería ser Setosa
